In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd 
#train = pd.read_csv('gdrive/My Drive/train_sample.csv')
#test = pd.read_csv('gdrive/My Drive/test_sample.csv')
#train = pd.read_csv('gdrive/My Drive/training_1perc_ranked_balanced_100K.csv')
#test = pd.read_csv('gdrive/My Drive/validation_1perc_ranked.csv')
train = pd.read_csv('gdrive/My Drive/training_set_VU_DM.csv')
test = pd.read_csv('gdrive/My Drive/test_set_VU_DM.csv')
print(len(train))
print(len(test))
#test = train.head(2000)

4958347
4959183


In [0]:
length = round(4958347 / 20)
train = train.head(length)

In [0]:
#n_list = [4,8,32,64,128,256,512,1024,2048,4096]
n = 512

In [5]:
print(train.columns)
print(test.columns)
#train = train.drop('rank_score', axis=1)
#test = test.drop('rank_score', axis=1)
#top = train.head(8000)
#bot = train.tail(8000)
#train = pd.concat([top,bot])
#top = test.head(2000)
#bot = test.tail(2000)
#test = pd.concat([top,bot])
#print(len(train))
#print(len(test))
#print(len(test['srch_id'].unique()))

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'

In [0]:
#train = train.head(16000)
#test = test.head(40000)
test_columns = test

In [0]:
train['month'] = pd.DatetimeIndex(train['date_time']).month
train = train.drop('date_time', axis=1)
test['month'] = pd.DatetimeIndex(test['date_time']).month
test= test.drop('date_time', axis=1)

In [0]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

columns = ['prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'price_usd',
           'orig_destination_distance', 'srch_booking_window', 'srch_query_affinity_score', 'visitor_hist_starrating', 'visitor_hist_adr_usd',
           'comp1_rate_percent_diff', 'comp2_rate_percent_diff','comp3_rate_percent_diff','comp4_rate_percent_diff','comp5_rate_percent_diff',
           'comp6_rate_percent_diff','comp7_rate_percent_diff','comp8_rate_percent_diff']

for column in columns:
  train[column] = remove_outlier(train, column)
  test[column] = remove_outlier(test, column)

In [0]:
def Vis(column, df):
    print('start')
    import matplotlib.pyplot as plt
    from matplotlib.patches import Patch
    # visualise discretization process for sepal lengths
    sorted_df = df.sort_values(by=[column])
    sorted_df = sorted_df[[column, 'colours']]
    sorted_df[column].plot(kind = 'bar', color = sorted_df['colours'], title = column)

    legend_elements = [Patch(facecolor='g', edgecolor='black',
                             label='booked'), Patch(facecolor='r', edgecolor='black',
                             label= 'not booked')]
    plt.legend(handles=legend_elements, loc='upper left')
    # The x-ticks were unreadible, so were removed. The plot is stil understandable (the instances are sorted in increasing order)
    plt.xlabel('Class of instance')
    # Set the y axis label of the current axis.
    plt.ylabel('Price usd')
    # Set a title of the current axes.
    #plt.title('Distribution of classes based on a sample of price_usd under 10000 usd')
    plt.xticks([], [])

visualisation = train.sample(1000)
visualisation = visualisation[visualisation['price_usd'] < 10000]
visualisation['colours'] = visualisation['booking_bool'].replace([0, 1], ['r', 'g'])

#Vis('price_usd', visualisation)

In [0]:
def NaN_percentage(df):

    # look for percentage of NaN values
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                     'percent_missing': percent_missing})
    missing_value_df = missing_value_df.reset_index(drop=True)
    print(missing_value_df)

#NaN_percentage(train)

In [0]:
#test = test.drop(['srch_id', 'prop_id', 'srch_destination_id', 'gross_bookings_usd', 'position', 'click_bool', 'booking_bool'], axis = 1)
#print(test.columns)
test = test.drop(['srch_id', 'prop_id', 'srch_destination_id'], axis = 1)

Replacing NaN values

In [0]:
def NaN_for_median(df, df_test):
    # replace NaN values
    df = df.fillna(df.median())
    df_test = df_test.fillna(df.median())
    return df, df_test

def NaN_for_mean(df, df_test):
    # replace NaN values
    df = df.fillna(df.mean())
    df_test = df_test.fillna(df.mean())
    return df, df_test

train, test = NaN_for_median(train, test)
#train, test= NaN_for_mean(train, test)

Outlier removal

Discretization

Calculate Gini

In [0]:
# calculate Gini 
def Gini(column, df):

    df_column = df.groupby([column, 'booking_bool']).size().reset_index(name ='Total_Amount')
    #print(df_column)
    categories = list(df_column[column].unique())
    #print(df_column)
    
    Gini_leafs = []
    sizes = []

    for i in range(len(categories)):
        #print('yes', column, categories[i])
        #print(df_column[(df_column[column] == categories[i]) & (df_column['booking_bool'] == 1)].Total_Amount)
        #print('no', column, categories[i])
        #print(df_column[(df_column[column] == categories[i]) & (df_column['booking_bool'] == 0)].Total_Amount)
        try:
            yes = int(df_column[(df_column[column] == categories[i]) & (df_column['booking_bool'] == 1)].Total_Amount)
        except:
            yes = 0
        try:
            no = int(df_column[(df_column[column] == categories[i]) & (df_column['booking_bool'] == 0)].Total_Amount)
        except:
            no = 0
        #print(yes, column, categories[i])
        #print(no, column, categories[i])
        prob_yes = yes / (yes + no)
        prob_no = no / (yes + no)
        Gini = 1 - (prob_yes)**2 - (prob_no)**2
        Gini_leafs.append(Gini)
        sizes.append(yes+no)

    Gini = 0
    for i in range(len(Gini_leafs)):
        new_score = (sizes[i] / sum(sizes)) * Gini_leafs[i]
        #print('new_score', new_score)
        Gini += new_score
        #print('Gini', Gini)

    return Gini


In [0]:
def calc_Gini(df):

    Gini_list = []

    for column in df.loc[:, df.columns != 'booking_bool'].columns:
        number = Gini(column, df)
        #print(column)
        #print(number)
        Gini_list.append(number)
    
    return Gini_list

Calculate feature importance

In [0]:
def feature_importance(df):
    FI_list = []

    from sklearn.datasets import make_classification
    from sklearn.ensemble import RandomForestClassifier
    from matplotlib import pyplot
    # define dataset
    X, y = df.loc[:, df.columns != 'booking_bool'], df['booking_bool']
    # define the model
    model = RandomForestClassifier()
    # fit the model
    model.fit(X, y)
    # get importance
    importance = model.feature_importances_
    # summarize feature importance
    for i,v in enumerate(importance):
        FI_list.append(v)
        #print(f'Feature %0d ({df.columns[i]}), Score: %.5f' % (i,v))
    # plot feature importance
    #pyplot.bar([x for x in range(len(importance))], importance)
    #pyplot.show()
    return FI_list

In [0]:
def importance_overview(df):
#print(len(df.loc[:, df.columns != 'booking_bool'].columns))
#print(len(Gini_list))
#print(len(FI_list))
    Gini_list = calc_Gini(df)
    FI_list = feature_importance(df) 

    d = {'feature':  df.loc[:, df.columns != 'booking_bool'].columns, 'Gini': Gini_list, 'FI': FI_list}
    df_Gini_FI = pd.DataFrame(data=d)
    df_Gini_FI['diff'] = df_Gini_FI['Gini'] - df_Gini_FI['FI']
    df_Gini_FI['both'] = df_Gini_FI['Gini'] * df_Gini_FI['FI']
    print(df_Gini_FI)

    print('Mean Gini:', df_Gini_FI['Gini'].mean())
    print('Mean FI:', df_Gini_FI['FI'].mean())
    print('Mean both:', df_Gini_FI['both'].mean())
    
    return df_Gini_FI

In [0]:
def topn(df_Gini_FI, n):
    topn_Gini = list(df_Gini_FI.sort_values('Gini').feature.head(n))
    topn_FI = list(df_Gini_FI.sort_values('FI').feature.head(n))
    topn_both = list(df_Gini_FI.sort_values('both').feature.head(n))
    d = {'Gini': topn_Gini, 'FI': topn_FI, 'both': topn_both}
    TopNValues= pd.DataFrame(data=d)
    TopNValues
    return TopNValues

In [0]:
#for_calc = test
#for_calc['booking_bool'] = test_columns['booking_bool']
# calculate Gini and Feature Importance (run first before second)
#df_Gini_FI = importance_overview(for_calc)
# calculate top n best for each score
#TopN = topn(df_Gini_FI, n = 5)
# select topn

In [0]:
# calculate top n best for each score
#TopN = topn(df_Gini_FI, n = 24)
#TopN
# select topn

In [0]:
#selection = TopN['Gini'].values + TopN['Gini'].values
#print(selection)
#output = []
#for x in selection:
#    if x not in output:
#        output.append(x)
#print(output)
#test = test[output]

Random Forest for classification

Random Forest for Ranking

In [0]:
def RankingRF(df,df_test,df_test_columns,n):

# Random Forest to order the submission file
    import time
    start_time = time.time()

    #print(df.columns)
    #print(df_test.head())
    #from sklearn.preprocessing import MinMaxScaler
    #scaler = MinMaxScaler()
    #df[:] = scaler.fit_transform(df.values)
    #df_test[:] = scaler.fit_transform(df_test.values)
    #df = df.reset_index()
    #df_test = df_test.reset_index()

    from sklearn.model_selection import train_test_split
    X=list(df_test.columns)  # Features
    y='booking_bool'  # Labels
    # Split dataset into training set and test set
    X_train, X_test, y_train = df[X], df_test[X], df[y]
    #Import Random Forest Model
    from sklearn.ensemble import RandomForestClassifier

    #Create a Gaussian Classifier
    clf=RandomForestClassifier(n_estimators=n)

    #Train the model using the training sets y_pred=clf.predict(X_test)
    clf.fit(X_train,y_train)
    y_pred=clf.predict_proba(X_test)

    #print(y_pred)

    #Import scikit-learn metrics module for accuracy calculation
    #from sklearn import metrics
    # Model Accuracy, how often is the classifier correct?
    #print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    chances_df = pd.DataFrame(clf.predict_proba(X_test), columns=['chance_not_booked', 'chance_booked'])

    df_test_columns['prob'] = chances_df['chance_booked']
    df_test_columns = df_test_columns[['srch_id', 'prop_id', 'prob']]
    df_test_columns = df_test_columns.sort_values('prob', ascending = False)
    #print(df_test_columns.head(100))
    #df_test_columns = df_test_columns.sort_values('srch_id')
    ranking = df_test_columns
    end_time = time.time()
    time = end_time - start_time
    #print('Time', time, 'seconds.')
    return ranking, time

In [0]:
#ranking, ranking_time = RankingRF(train,test,test_columns,n)
ranking, ranking_time = RankingRF(train,test,test_columns,n)
submission = ranking

In [23]:
submission_byprob = submission
#submission_bysrchid = submission.sort_values('srch_id')
#submission_bysrchid = submission_bysrchid.sort_values('prob', ascending = False)
#submission_bysrchid = submission_bysrchid.sort_values('srch_id', ascending = True)
submission_bysrchid = submission.sort_values(['srch_id','prob'],ascending=[True,False])
print(submission_byprob.head())
print(submission_bysrchid.head())
submission_byprob = submission_byprob[['srch_id', 'prop_id']]
submission_bysrchid= submission_bysrchid[['srch_id', 'prop_id']]
print(submission_byprob.head())
print(submission_bysrchid.head())
from google.colab import files
#results.to_csv(r'RFtrees2.csv', sep=',', encoding='utf-8', index = True)
#results2.to_csv(r'RFtreeshead2.csv', sep=',', encoding='utf-8', index = True)
#files.download('RFtrees2.csv')
#files.download('RFtreeshead2.csv')

         srch_id  prop_id      prob
4920670   330253   100969  0.697266
1688979   113596   106652  0.675781
3780737   254047    36030  0.666016
4458491   299351   110695  0.666016
2957047   199082   100969  0.656250
    srch_id  prop_id      prob
23        1    99484  0.101562
8         1    50162  0.093750
21        1    94729  0.085938
9         1    54937  0.078125
4         1    24194  0.074219
         srch_id  prop_id
4920670   330253   100969
1688979   113596   106652
3780737   254047    36030
4458491   299351   110695
2957047   199082   100969
    srch_id  prop_id
23        1    99484
8         1    50162
21        1    94729
9         1    54937
4         1    24194


In [0]:
from google.colab import files
submission_bysrchid.to_csv(r'RFtrees512srchid_end.csv', sep=',', encoding='utf-8', index = False)
submission_byprob.to_csv(r'RFtrees512prob_end.csv', sep=',', encoding='utf-8', index = False)
files.download('RFtrees512srchid_end.csv')
files.download('RFtrees512prob_end.csv')

In [0]:
def evaluation(ranking, test_column):

  val = ranking
  val[['srch_id', 'prop_id', 'click_bool', 'booking_bool']] = test_columns[['srch_id', 'prop_id', 'click_bool', 'booking_bool']]
  val = val[['srch_id', 'prop_id', 'click_bool', 'booking_bool', 'prob']]
  val['rank'] = val['click_bool'] + (val['booking_bool'] * 4)

  val_check = test_columns[['srch_id', 'prop_id', 'click_bool', 'booking_bool']]
  val_check['rank'] = val['rank']

  val = val.sort_values('prob', ascending = False)
  val_check = val.sort_values('rank', ascending = False)

  import statistics as s
  #print(s.mean(accuracies))

  val_gold = val_check[['srch_id', 'rank']]
  val_pred = val[['srch_id', 'rank']]

  import numpy as np

  def dcgm38(sequence):
      score = 0
      i = 1
      for _, s in sequence.iterrows():
          score += (2**s['rank']-1)/np.log2(i+1)
          i += 1
      return score

  # with sorting
  srch_ids = np.unique(val_gold['srch_id'])
  dcgm_max = np.empty(len(srch_ids))
  for i, s in enumerate(srch_ids):
      dcgm_max[i] = dcgm38(val_gold.loc[val_gold['srch_id']==s])

  srch_ids = np.unique(val_pred['srch_id'])
  dcgm_pred = np.empty(len(srch_ids))
  for i, s in enumerate(srch_ids):
      dcgm_pred[i] = dcgm38(val_pred.loc[val_pred['srch_id']==s])

  #print(dcgm_max)
  #print(dcgm_max)

  maximum = np.average(dcgm_max)
  prediction_score = np.average(dcgm_pred)
  print(maximum, prediction_score)
  grouped = np.mean(np.divide(prediction_score, maximum))
  #print('mean NDCG (grouped by "srch_id"):', np.mean(np.divide(prediction_score, maximum)))

  #without sorting
  dcgm_max = dcgm38(val_gold)
  #print(val_gold)
  #print(dcgm_max)

  #rch_ids = np.unique(val_pred['srch_id'])
  #cgm_pred = np.empty(len(srch_ids))
  #or i, s in enumerate(srch_ids):
  dcgm_pred = dcgm38(val_pred)

  #print(dcgm_max)
  #print(dcgm_max)

  #maximum = np.average(dcgm_max)
  #prediction_score = np.average(dcgm_pred)
  maximum = np.average(dcgm_max)
  prediction_score = np.average(dcgm_pred)
  print(maximum, prediction_score)
  not_grouped = np.mean(np.divide(prediction_score, maximum))
  #print('mean NDCG (only sorted by "prob"):', np.mean(np.divide(prediction_score, maximum)))
  return grouped, not_grouped

In [0]:

time_list = []
grouped_list = []
not_grouped_list= []

print('ok1')

for n in n_list:
  print('ok2')
  ranking, ranking_time = RankingRF(train,test,test_columns,n)
  print('Trees:', n)
  print('Time:', ranking_time)
  time_list.append(ranking_time)
  grouped, not_grouped = evaluation(ranking, test_columns)
  print('Grouped, not_grouped:', grouped, not_grouped)
  print('grouped')
  grouped_list.append(grouped)
  print('not_grouped')
  not_grouped_list.append(not_grouped)

outcomes = {'trees': n_list, 'time': time_list, 'NDCG sorted': grouped_list, 'NDCG not sorted': not_grouped_list}
results = pd.DataFrame(data=outcomes)

In [0]:
print(results)
print(results[results['NDCG sorted'] == results['NDCG sorted'].max()])
print(results[results['NDCG not sorted'] == results['NDCG not sorted'].max()])

In [0]:
print(results[results['NDCG sorted'] == results['NDCG sorted'].max()])
print(results[results['NDCG not sorted'] == results['NDCG not sorted'].max()])

In [0]:
x = results['trees']
y1 = results['NDCG sorted']
y2= results['NDCG not sorted']

import matplotlib.pyplot as plt
plt.plot(x, y1, label = "considering srch_id")
plt.plot(x, y2, label = "ignoring srch_id")
plt.xlabel('Amount of trees')
# Set the y axis label of the current axis.
plt.ylabel('NDCG')
# Set a title of the current axes.
plt.title('NDCG for each amount of trees ')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()


In [0]:
results2 = results.head(7)
print(results2[results2['NDCG sorted'] == results2['NDCG sorted'].max()])
print(results2[results2['NDCG not sorted'] == results2['NDCG not sorted'].max()])


In [0]:
from google.colab import files
#results.to_csv(r'RFtrees2.csv', sep=',', encoding='utf-8', index = True)
#results2.to_csv(r'RFtreeshead2.csv', sep=',', encoding='utf-8', index = True)
#files.download('RFtrees2.csv')
#files.download('RFtreeshead2.csv')

Options